# <p style="text-align:center;font-size:150%;font-family:Roboto;background-color:#a04070;border-radius:50px;font-weight:bold;margin-bottom:0">Neural Style Transfer</p>

<p style="font-family:Roboto;font-size:120%;color:#a04070;">In this Notebook, I will build a Neural Style Transfer model from scratch by using a pretrained model VGG19 which is considered as one of the best computer vision architecture used for the transfer learning. In this task our goal is to transfer style of one image known as Style Image to the other known as Content Image. Pre trained model VGG19's different layers help us to identify the style representations present in the Style image and then we use it to apply on our content image.</p> 

<!-- <a id='top'></a> -->
<div class="list-group" id="list-tab" role="tablist">
<p style="background-color:#a04070;font-family:Roboto;font-size:160%;text-align:center;border-radius:50px;">TABLE OF CONTENTS</p>   
    
* [1. Importing Libraries](#1)
    
* [2. Loading the model and Defining layers](#2)
    
* [3. Cost Functions](#3)
    
* [4. Getting Convolved Outputs and Visualizing Them](#4) 
    
* [5. Style Transfer Step](#5) 
      
* [6. Visualizing and Saving the Generated Image](#6)
    
* [7. Some points from the paper](#7)
    
* [8. End](#8)


<a id="1"></a>
# <p style="background-color:#a04070;font-family:Roboto;font-size:120%;text-align:center;border-radius:50px;margin-bottom:0">IMPORTING LIBRARIES</p>
<p style="font-family:Roboto;font-size:140%;margin-up:0;color:#a04070;">These will help us to plot the style and the content image along with the intermediate image generated in the style tranfer step.</p>
    

In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

<a id="2"></a>
# <p style="background-color:#a04070;font-family:Roboto;font-size:120%;text-align:center;border-radius:50px;margin-bottom:0;padding-down:0;">Loading Model and Defining Layers</p>  
<ul style="color:#a04070;font-family:Roboto;font-size:140%;">
<li>Higher layers in the network capture the high level content in terms of objects and their arrangement but do not constraint to the exact pixel values of the reconstruction that is why we reffered last layer of the convolution as the content layer.</li>
    
<li>The shallow layers of the network are more capable of producing the exact pixel values of the original image that is why we reffered them as the style layers.</li>
</ul>

In [2]:
img_size = 600
vgg = tf.keras.applications.VGG19(include_top=False,
                                  input_shape=(img_size, img_size, 3),
                                  weights='imagenet')

vgg.trainable = False

In [3]:
for layer in vgg.layers:
    print(layer.name)

In [4]:
# By using the above arguments given above.
STYLE_LAYERS = [
    ('block1_conv1', 0.2),
    ('block2_conv1', 0.2),
    ('block3_conv1', 0.2),
    ('block4_conv1', 0.2),
    ('block5_conv1', 0.2)]

content_layer = [('block5_conv4', 2)]

In [5]:
def get_layer_outputs(vgg, layer_names):
    outputs = [vgg.get_layer(layer[0]).output for layer in layer_names]

    model = tf.keras.Model(vgg.input, outputs)
    return model

vgg_model_outputs = get_layer_outputs(vgg, STYLE_LAYERS + content_layer)

# <p style="color:#a04070;font-family:Roboto;font-size:80%;margin-bottom:0">These are some utility functions which will help us to visualize the generated images throughout the style transfer.</p> 

In [10]:
def clip_0_1(image):
    #Truncate all the pixels in the tensor to be between 0 and 1
    
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

def tensor_to_image(tensor):
    #Converts the given tensor into a PIL image

    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return Image.fromarray(tensor)

<a id="4"></a>
# <p style="background-color:#a04070;font-family:Roboto;font-size:120%;text-align:center;border-radius:50px;margin-bottom:0;padding-down:0;">Getting Convolved Outputs and Visualizing Them</p>  
<p style="color:#a04070;font-family:Roboto;font-size:140%;">Each layer of the convolution provide us with convolved output of the input image with higher understanding of the style representations in the given image. We will try to visualize them for some of the layers upto which it is understandable for the human eye.</p>

In [6]:
content_image = np.array(Image.open("../input/images/Taj Mahal.jpg").resize((img_size, img_size)))
content_image = tf.constant(np.reshape(content_image, ((1,) + content_image.shape)))

style_image =  np.array(Image.open("../input/images/Style7.jpg").resize((img_size, img_size)))
style_image = tf.constant(np.reshape(style_image, ((1,) + style_image.shape)))

plt.figure(figsize=(15,10))

plt.subplot(1,2,1)
plt.title(str(style_image.shape))
plt.imshow(style_image[0])

plt.subplot(1,2,2)
plt.title(str(content_image.shape))
plt.imshow(content_image[0])
plt.show()

In [7]:
generated_image = tf.Variable(tf.image.convert_image_dtype(content_image, tf.float32))
noise = tf.random.uniform(tf.shape(generated_image), -0.25, 0.25)
generated_image = tf.add(generated_image, noise)
generated_image = tf.clip_by_value(generated_image, clip_value_min=0.0, clip_value_max=1.0)

plt.figure(figsize=(15,5))
plt.title(str(generated_image.shape))
plt.imshow(generated_image.numpy()[0])
plt.show()

In [8]:
preprocessed_content =  tf.Variable(tf.image.convert_image_dtype(content_image, tf.float32))
a_C = vgg_model_outputs(preprocessed_content)

preprocessed_style =  tf.Variable(tf.image.convert_image_dtype(style_image, tf.float32))
a_S = vgg_model_outputs(preprocessed_style)

<p style="color:#a04070;font-family:Roboto;font-size:140%;">This is what the outputs of the convolved layer looks like.</p>

In [11]:
# Convolutions from the first layer
for j in range(4):
    plt.subplots(figsize=(10,10))
    print('Convolutions from layer number : ', str(j))
    for i in range(7):
        image = tensor_to_image(a_S[j][0, :, :, i * 10 +1])

        plt.subplot(3,3,i+1)
        plt.imshow(image)
    plt.tight_layout()
    plt.show()

<a id="3"></a>
# <p style="background-color:#a04070;font-family:Roboto;font-size:120%;text-align:center;border-radius:50px;margin-bottom:0;padding-down:0;">Cost Functions</p>  
<p style="color:#a04070;font-family:Roboto;font-size:120%;">These are the cost functions to compute cost for the content layer as well as the style layers also as the loss function we define try to optimize the content cost as well as style cost we can emphasize on one by increasing the weights of it in the total cost function.</p>

In [12]:
def compute_content_cost(content_output, generated_output):
    #a_C --  (1, n_H, n_W, n_C), hidden layer activations representing content of the image C 
    #a_G --  (1, n_H, n_W, n_C), hidden layer activations representing content of the image G
    
    a_C = content_output[-1]
    a_G = generated_output[-1]
    
    _, n_H, n_W, n_C = a_G.get_shape()
    
    a_C_unrolled = tf.transpose(tf.reshape(a_C, shape=[n_H * n_W, n_C]))
    a_G_unrolled = tf.transpose(tf.reshape(a_G, shape=[n_H * n_W, n_C]))
    
    J_content = tf.reduce_sum(tf.reduce_sum(tf.square(tf.subtract(a_C_unrolled, a_G_unrolled)), axis=1), axis=0)
    J_content = (1/(4 * n_H * n_W * n_C)) * J_content
    
    return J_content

<p style="color:#a04070;font-family:Roboto;font-size:140%;">
While calculating the style matrix we take the correlation between different layers by different filters because this helps to obtain a stationary, multi scale representations of the input image, which captures it's texture information but not the global arrangement.
</p>

In [13]:
def compute_layer_style_cost(a_S, a_G):
    #a_S -- (1, n_H, n_W, n_C), hidden layer activations representing style of the image S 
    #a_G -- (1, n_H, n_W, n_C), hidden layer activations representing style of the image G
    
    _, n_H, n_W, n_C = a_G.get_shape()
    
    a_S = tf.transpose(tf.reshape(a_S, shape=[n_H * n_W, n_C]))
    a_G = tf.transpose(tf.reshape(a_G, shape=[n_H * n_W, n_C]))

    
#     Style or the Gram Matrix calculation.
    GS = tf.matmul(a_S, tf.transpose(a_S))
    GG = tf.matmul(a_G, tf.transpose(a_G))

    J_style_layer = tf.reduce_sum(tf.reduce_sum(tf.square(tf.subtract(GS, GG)), axis=1), axis=0)
    J_style_layer = (1/(4 * n_H * n_W * n_C * n_H * n_W * n_C)) * J_style_layer
    
    return J_style_layer

In [14]:
def compute_style_cost(style_image_output, generated_image_output, STYLE_LAYERS=STYLE_LAYERS):

    J_style = 0

    # The last element of the array contains the content layer image, which must not be used.
    a_S = style_image_output[:-1]
    a_G = generated_image_output[:-1]

    for i, weight in zip(range(len(a_S)), STYLE_LAYERS):  
        J_style_layer = compute_layer_style_cost(a_S[i], a_G[i])
        J_style += weight[1] * J_style_layer

    return J_style

In [15]:
@tf.function()
def total_cost(J_content, J_style, alpha = 30, beta = 10):
    J = alpha * J_content + beta * J_style
    return J

<a id="5"></a>
# <p style="background-color:#a04070;font-family:Roboto;font-size:120%;text-align:center;border-radius:50px;margin-bottom:0;padding-down:0;">Style Transfer Step</p>  
<p style="color:#a04070;font-family:Roboto;font-size:140%;">By using the GradientTape we will apply gradient descent on our algorithm to reduce the loss defined between the generated image and the content image + style image is reduced to minimum. </p>

In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.025)

@tf.function()
def train_step(generated_image):
    with tf.GradientTape() as tape:
        
        a_G = vgg_model_outputs(generated_image)
        
        J_style = compute_style_cost(a_S, a_G)
        J_content = compute_content_cost(a_C, a_G)

        J = total_cost(J_content, J_style)
        
    grad = tape.gradient(J, generated_image)

    optimizer.apply_gradients([(grad, generated_image)])
    generated_image.assign(clip_0_1(generated_image))

In [17]:
generated_image = tf.Variable(generated_image)

epochs = 2501

for j in range(epochs//250 + 1):
    for i in range(250):
        train_step(generated_image)

    image = tensor_to_image(generated_image)
    plt.title(f"Epoch {j*250} ")
    plt.imshow(image)
    plt.show()

<a id="6"></a>
# <p style="background-color:#a04070;font-family:Roboto;font-size:120%;text-align:center;border-radius:50px;margin-bottom:0;padding-down:0;">Visualizing and Saving the Generated Image</p>  
<p style="color:#a04070;font-family:Roboto;font-size:140%;">Observe what are the main features of the style image has been transferred to our content image along with preserving the content.</p>

In [18]:
plt.figure(figsize=(16, 4))

plt.subplot(1, 3, 1)
plt.imshow(content_image[0])
plt.title('Content image')

plt.subplot(1, 3, 2)
plt.imshow(style_image[0])
plt.title('Style image')

plt.subplot(1, 3, 3)
plt.imshow(generated_image[0])
plt.title('Generated image')
plt.show()

In [19]:
image = tensor_to_image(generated_image)

plt.figure(figsize=(15,5))
plt.imshow(image)
plt.show() 

image.save("Taj + Style3.jpg")

<a id="7"></a>
# <p style="background-color:#a04070;font-family:Roboto;font-size:120%;text-align:center;border-radius:50px;margin-bottom:0;padding-bottom:0;">Some Points from <a href='https://arxiv.org/abs/1508.06576' style='color:Gray;'>The Paper<a/></p>  
<p style="color:#a04070;font-family:Roboto;font-size:140%;">Question 1: Why we have to consider all the earlier layers to get the style representations instead of just taking some of the starting layers?</p>
<p style="color:#a04070;font-family:Roboto;font-size:140%;">Answer: As the per the paper mentioned above, matching the style representations up to the higher layers leads to a smoother and more continous visual experience.<br> Hence, visually most appealing images are generated by matching the style representations up to the highest layers in the network.</p>

<p style="color:#399983;font-family:Roboto;font-size:140%;">Question 2: Why we use a pretrained model instead of defining one and training it?</p>
<p style="color:#399983;font-family:Roboto;font-size:140%;">Answer: One of the obvious reason is that we don't have that much resources to train a high performing model.<br> We use a pretrained network for object detection because they are invariant to all the image variation that preserves object identity.</p>

<a id="8"></a>
# <p style="background-color:#a04070;font-family:Roboto;font-size:120%;text-align:center;border-radius:50px;margin-bottom:0;padding-bottom:0;">The End</p>
<ul><li style="color:#140803;font-family:Roboto;font-size:120%;margin-bottom:0;padding-bottom:0;">Inspired from the awesome use of markdown by <a href='https://www.kaggle.com/code/karnikakapoor/art-by-gan'>Karnika Kapoor</a> to make the notebooks more readable, in this notebook I had tried to learn the same from her notebook and applied it here for the very first time.</li>
<li style="color:#140803;font-family:Roboto;font-size:120%;margin-bottom:0;padding-bottom:0;">Images I used in this notebook are taken from unsplash.com.</li>

<li style="color:#140803;font-family:Roboto;font-size:120%;margin-bottom:0;padding-bottom:0;">At last I would like to suggest that copy this notebook and try to play around with the weights and different style and the content images.</li></ul>